In [ ]:
# First, completely clean up existing installations
!pip uninstall -y torch torchvision torchaudio transformers trl accelerate peft bitsandbytes

# Install specific compatible versions
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.36.2  # Version known to work with trl 0.7.10

In [ ]:
!pip install accelerate==0.27.2
!pip install peft==0.7.1
!pip install trl==0.7.10

In [ ]:
!pip install bitsandbytes==0.41.3
!pip install datasets==2.15.0

In [ ]:
!pip install sentencepiece protobuf huggingface_hub

# Restart runtime after installation
import os
os._exit(00)  # This will restart the runtime in Colab

In [ ]:
# ShadCN Component Generator with LLaMA 3.1 Fine-tuning
# Updated version with fixed package versions and error handling

import subprocess
import sys
import os
import warnings
import platform
import json
import gc
from pathlib import Path
warnings.filterwarnings("ignore")

# --------------------------
# Package Installation Setup
# --------------------------

def check_gpu_info():
    """Check GPU and CUDA information"""
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        if result.returncode == 0:
            print("🎮 GPU Information:")
            lines = result.stdout.split('\n')
            for line in lines:
                if 'CUDA Version' in line:
                    print(f"   {line.strip()}")
                elif 'GeForce' in line or 'Tesla' in line or 'Quadro' in line or 'RTX' in line:
                    print(f"   GPU: {line.split('|')[1].strip()}")
            return True
    except:
        print("⚠️ No NVIDIA GPU detected or nvidia-smi not available")
        return False

def install_package(package):
    """Install package with error handling"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--no-cache-dir"])
        print(f"✅ Installed: {package}")
        return True
    except Exception as e:
        print(f"❌ Failed to install {package}: {e}")
        return False

def uninstall_package(package):
    """Uninstall package safely"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", package])
        print(f"🗑️ Uninstalled: {package}")
    except:
        pass

def fix_torch_installation():
    """Fix PyTorch installation with proper CUDA support"""
    print("🔧 Fixing PyTorch installation...")
    
    # Check if we're in Colab
    try:
        import google.colab
        IN_COLAB = True
        print("📍 Detected Google Colab environment")
    except:
        IN_COLAB = False
        print("📍 Local environment detected")
    
    # Check GPU availability
    has_gpu = check_gpu_info()
    
    # Clean installation
    print("🧹 Cleaning existing PyTorch installation...")
    packages_to_remove = ["torch", "torchvision", "torchaudio", "bitsandbytes", "transformers", "accelerate", "peft", "trl"]
    for pkg in packages_to_remove:
        uninstall_package(pkg)
    
    # Install PyTorch based on environment
    if IN_COLAB:
        # Colab typically has CUDA 11.8 or 12.1
        print("📦 Installing PyTorch for Colab...")
        success = install_package("torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118")
        if not success:
            # Fallback to CPU version
            print("⚠️ GPU version failed, installing CPU version...")
            install_package("torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu")
    else:
        # Local installation - try to detect CUDA version
        if has_gpu:
            print("📦 Installing PyTorch with CUDA support...")
            success = install_package("torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118")
            if not success:
                success = install_package("torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")
            if not success:
                print("⚠️ CUDA versions failed, installing CPU version...")
                install_package("torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu")
        else:
            print("📦 Installing CPU-only PyTorch...")
            install_package("torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu")

def install_ml_packages():
    """Install ML packages with version compatibility"""
    print("📦 Installing compatible ML packages...")
    
    # Core packages with compatible versions
    packages = [
        "transformers==4.36.2",
        "accelerate==0.27.2",
        "peft==0.7.1",
        "trl==0.7.10",
        "datasets==2.15.0",
        "sentencepiece",
        "protobuf",
        "huggingface_hub",
    ]
    
    for package in packages:
        install_package(package)
    
    # Install bitsandbytes carefully
    try:
        import torch
        if torch.cuda.is_available():
            print("📦 Installing bitsandbytes for GPU...")
            install_package("bitsandbytes==0.41.3")
        else:
            print("📦 Installing bitsandbytes for CPU...")
            install_package("bitsandbytes-cpu")
    except:
        print("⚠️ Installing basic bitsandbytes...")
        install_package("bitsandbytes")

# --------------------------
# Initial Setup and Checks
# --------------------------

# Fix installation
fix_torch_installation()
install_ml_packages()

print("✅ Installation completed! Testing imports...")

# Test imports with better error handling
try:
    import torch
    print(f"✅ PyTorch {torch.__version__} imported successfully")
    print(f"🔍 CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"🔍 CUDA version: {torch.version.cuda}")
        print(f"🔍 GPU count: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"🔍 GPU {i}: {torch.cuda.get_device_name(i)}")
except Exception as e:
    print(f"❌ PyTorch import failed: {e}")
    print("💡 Try restarting runtime and running again")
    sys.exit(1)

# Import other libraries after installation
try:
    from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
    print("✅ Transformers imported")
except Exception as e:
    print(f"❌ Transformers import failed: {e}")
    sys.exit(1)

try:
    from datasets import Dataset
    print("✅ Datasets imported")
except Exception as e:
    print(f"❌ Datasets import failed: {e}")
    sys.exit(1)

try:
    from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
    print("✅ PEFT imported")
except Exception as e:
    print(f"❌ PEFT import failed: {e}")
    sys.exit(1)

try:
    from trl import SFTTrainer
    print("✅ TRL imported")
except Exception as e:
    print(f"❌ TRL import failed: {e}")
    sys.exit(1)

# Try to import bitsandbytes
try:
    import bitsandbytes as bnb
    QUANTIZATION_AVAILABLE = True
    print("✅ BitsAndBytes available for quantization")
except Exception as e:
    QUANTIZATION_AVAILABLE = False
    print(f"⚠️ BitsAndBytes not available: {e}")

# --------------------------
# Configuration
# --------------------------

MODEL_CONFIGS = {
    "llama3.1-8b": "meta-llama/Llama-3.1-8B-Instruct",
    "llama3.1-8b-4bit": "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "phi3-mini": "microsoft/Phi-3-mini-4k-instruct",
    "qwen2-7b": "Qwen/Qwen2-7B-Instruct",
}

# Choose model based on available resources
if torch.cuda.is_available() and torch.cuda.get_device_properties(0).total_memory > 12e9:  # >12GB VRAM
    MODEL_NAME = MODEL_CONFIGS["llama3.1-8b-4bit"]
    MAX_SEQ_LENGTH = 2048
    BATCH_SIZE = 1
    print("🎯 Using LLaMA 3.1 8B (4-bit) - High-end setup")
elif torch.cuda.is_available():
    MODEL_NAME = MODEL_CONFIGS["phi3-mini"]
    MAX_SEQ_LENGTH = 1024
    BATCH_SIZE = 1
    print("🎯 Using Phi-3 Mini - Medium setup")
else:
    MODEL_NAME = MODEL_CONFIGS["phi3-mini"]
    MAX_SEQ_LENGTH = 512
    BATCH_SIZE = 1
    print("🎯 Using Phi-3 Mini CPU mode - Basic setup")

# Training configuration
OUTPUT_DIR = "./shadcn-component-generator"
GRADIENT_ACCUMULATION_STEPS = 4
MAX_STEPS = 100  # Reduced for testing
LEARNING_RATE = 2e-4

print(f"📊 Configuration:")
print(f"   Model: {MODEL_NAME}")
print(f"   Max sequence length: {MAX_SEQ_LENGTH}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Max steps: {MAX_STEPS}")

# --------------------------
# Dataset and Training Setup
# --------------------------

def create_sample_dataset():
    """Create a comprehensive sample dataset for ShadCN components"""
    components_data = [
        {
            "component_name": "button",
            "description": "Create a versatile button component with multiple variants",
            "registry_json": {
                "name": "button",
                "type": "component",
                "dependencies": ["class-variance-authority", "clsx"],
                "props": {
                    "variant": {
                        "type": "enum",
                        "values": ["default", "destructive", "outline", "secondary", "ghost", "link"],
                        "default": "default"
                    },
                    "size": {
                        "type": "enum", 
                        "values": ["default", "sm", "lg", "icon"],
                        "default": "default"
                    }
                },
                "css": {
                    "base": "inline-flex items-center justify-center rounded-md text-sm font-medium ring-offset-background transition-colors focus-visible:outline-none focus-visible:ring-2 focus-visible:ring-ring focus-visible:ring-offset-2 disabled:pointer-events-none disabled:opacity-50",
                    "variants": {
                        "default": "bg-primary text-primary-foreground hover:bg-primary/90",
                        "destructive": "bg-destructive text-destructive-foreground hover:bg-destructive/90",
                        "outline": "border border-input bg-background hover:bg-accent hover:text-accent-foreground",
                        "secondary": "bg-secondary text-secondary-foreground hover:bg-secondary/80",
                        "ghost": "hover:bg-accent hover:text-accent-foreground",
                        "link": "text-primary underline-offset-4 hover:underline"
                    },
                    "sizes": {
                        "default": "h-10 px-4 py-2",
                        "sm": "h-9 rounded-md px-3",
                        "lg": "h-11 rounded-md px-8", 
                        "icon": "h-10 w-10"
                    }
                }
            }
        },
        # ... (rest of your dataset components remain the same)
    ]
    
    return components_data

def format_training_prompt(example):
    """Format training examples for the model"""
    component_name = example["component_name"]
    description = example["description"]
    registry_json = example["registry_json"]
    
    prompt = f"""Create a ShadCN/UI component registry for: {component_name}

Description: {description}

Component Registry JSON:
{json.dumps(registry_json, indent=2)}"""
    
    return {"text": prompt}

def load_model_and_tokenizer():
    """Load model and tokenizer with error handling"""
    try:
        print(f"🔄 Loading model: {MODEL_NAME}")
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
        
        # Set padding token
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right"
        
        print("✅ Tokenizer loaded")
        
        # Configure quantization if available
        model_kwargs = {
            "trust_remote_code": True,
            "torch_dtype": torch.float16 if torch.cuda.is_available() else torch.float32,
        }
        
        if QUANTIZATION_AVAILABLE and torch.cuda.is_available():
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True
            )
            model_kwargs["quantization_config"] = bnb_config
            model_kwargs["device_map"] = "auto"
            print("✅ Using 4-bit quantization")
        
        # Load model
        model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, **model_kwargs)
        print("✅ Model loaded")
        
        # Apply LoRA if using quantization
        if QUANTIZATION_AVAILABLE and torch.cuda.is_available():
            model = prepare_model_for_kbit_training(model)
            
            peft_config = LoraConfig(
                r=16,
                lora_alpha=32,
                target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
                lora_dropout=0.05,
                bias="none",
                task_type="CAUSAL_LM"
            )
            
            model = get_peft_model(model, peft_config)
            print("✅ LoRA configuration applied")
        
        return model, tokenizer
        
    except Exception as e:
        print(f"❌ Failed to load model: {e}")
        raise

def train_model():
    """Main training function"""
    try:
        # Load model and tokenizer
        model, tokenizer = load_model_and_tokenizer()
        
        # Create dataset
        print("📊 Creating training dataset...")
        sample_data = create_sample_dataset()
        formatted_data = [format_training_prompt(example) for example in sample_data]
        
        # Convert to Dataset object
        dataset = Dataset.from_list(formatted_data)
        print(f"✅ Dataset created with {len(dataset)} examples")
        
        # Training arguments
        training_args = TrainingArguments(
            output_dir=OUTPUT_DIR,
            per_device_train_batch_size=BATCH_SIZE,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
            learning_rate=LEARNING_RATE,
            max_steps=MAX_STEPS,
            logging_steps=10,
            save_steps=50,
            warmup_steps=10,
            optim="adamw_hf",
            lr_scheduler_type="cosine",
            fp16=torch.cuda.is_available(),
            dataloader_drop_last=True,
            remove_unused_columns=False,
            report_to="none",
            seed=42,
        )
        
        # Create trainer
        trainer = SFTTrainer(
            model=model,
            tokenizer=tokenizer,
            train_dataset=dataset,
            dataset_text_field="text",
            max_seq_length=MAX_SEQ_LENGTH,
            args=training_args,
            packing=False,
        )
        
        print("🚀 Starting training...")
        trainer.train()
        
        # Save model
        trainer.save_model(OUTPUT_DIR)
        tokenizer.save_pretrained(OUTPUT_DIR)
        
        print(f"✅ Training completed! Model saved to {OUTPUT_DIR}")
        return True
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
        return False

def test_generation():
    """Test component generation"""
    try:
        print("🧪 Testing component generation...")
        
        # Load trained model if available
        model_path = OUTPUT_DIR if Path(OUTPUT_DIR).exists() else MODEL_NAME
        
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None
        )
        
        # Test prompts
        test_prompts = [
            "Create a badge component with different variants and sizes",
            "Create a tooltip component with smooth animations",
            "Create a progress bar component with customizable styling"
        ]
        
        for prompt in test_prompts:
            print(f"\n📝 Prompt: {prompt}")
            
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
            if torch.cuda.is_available():
                inputs = {k: v.to(model.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=300,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                )
            
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = generated_text[len(prompt):].strip()
            
            print("Generated:")
            print(response[:200] + "..." if len(response) > 200 else response)
            print("-" * 50)
            
    except Exception as e:
        print(f"❌ Generation test failed: {e}")

# --------------------------
# Main Execution
# --------------------------

if __name__ == "__main__":
    print("\n" + "="*60)
    print("🎨 SHADCN COMPONENT GENERATOR WITH LLAMA 3.1")
    print("="*60)
    
    # Clear GPU memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    # Run training
    success = train_model()
    
    if success:
        print("\n🎉 Training completed successfully!")
        test_generation()
    else:
        print("\n❌ Training failed. Check the error messages above.")
    
    print(f"\n✨ ShadCN Generator ready!")
    print(f"📁 Model saved in: {OUTPUT_DIR}")
    print("🚀 Ready to generate ShadCN components!")